### launching of SPs



In [1]:
from datetime import timedelta
from decimal import Decimal
from tqdm import tqdm
import pandas as pd

import pyodbc
from sql.config import SQL_SERVER, DB_NAME
import sql.naming_convention as nc
from sql.sql_requests import SQL_Communicator

In [2]:
def format_time_delta(duration: timedelta):
    # Get the total seconds
    total_seconds = duration.total_seconds()
    # days = duration.days
    hours = duration.seconds // 3600  # Convert seconds to hours
    minutes = (duration.seconds // 60) % 60  # Convert remaining seconds to minutes
    seconds = duration.seconds % 60
    if hours > 0:
        return f'{hours:02d}:{minutes:02d}:{int(seconds):02d}'
    if minutes > 0:
        return f'{minutes:02d}:{int(seconds):02d}'
    return f'{total_seconds:4.2f}'

In [3]:
entity_names =[
"Professional_Compensation",
"Professional_Person",
"Professional_BiographyPerson",
"Professional_Professional",
"Professional_ProfessionalCoverage",
"Professional_ProToProFunction",
]

durations = []
with SQL_Communicator() as sc:
    for en in tqdm(entity_names):
        print(f'flow for target entity {en}..')
        en2 = nc.default_rename(en)
        sp_names = [nc.pull_sp_name(en2), nc.merge_sp_name(en2), nc.merge_direct_sp_name(en2)]
        stage_tbl = nc.stg_table_name(en)
        trg_tbl = nc.table_name(nc.default_rename(en))
        ent_durs, flow_td = [en2], timedelta()  
        for i, sp_name in enumerate(sp_names):
            exec_stm = f"EXEC {sp_name}"
            print(f'- {exec_stm}...')
            try:
                td = sc.get_execution_metrics(exec_stm)
                ent_durs.append(format_time_delta(td))
                if i < 2:
                    flow_td +=td
                if  i == 1:
                    ent_durs.append(format_time_delta(flow_td))
                print(f'  - duration {format_time_delta(td)}')
                if i == 0:  ## pull sp
                    rows_count, table_size = sc.get_table_size(stage_tbl)
                    print(f'rows {rows_count}, stage table size: {table_size}')
                sc.get_execution_metrics(f'TRUNCATE table {trg_tbl}')
            except Exception as ex:
                print(f'shit happened {ex}')
        rows_count = rows_count /1000
        velocity = table_size / Decimal(flow_td.total_seconds()) if flow_td.total_seconds() else 0 
        ent_durs.extend([f'{rows_count:7.2f}' , f'{table_size:7.2f}', f'{velocity:7.2f}'])
        durations.append(ent_durs)
        # ent_durs.append(sum(ent_durs))


connecting...
Connection successful!


  0%|          | 0/6 [00:00<?, ?it/s]

flow for target entity Professional_Compensation..
- EXEC dbo.PullData_Professional_Compensation2_prc...
  - duration 02:44
rows 48222280, stage table size: 3795.203125
- EXEC dbo.MergeData_Professional_Compensation2_prc...
  - duration 05:52
- EXEC dbo.MergeDataDirect_Professional_Compensation2_prc...


 17%|█▋        | 1/6 [12:52<1:04:22, 772.41s/it]

  - duration 04:15
flow for target entity Professional_Person..
- EXEC dbo.PullData_Professional_Person2_prc...
  - duration 40.43
rows 5959980, stage table size: 361.015625
- EXEC dbo.MergeData_Professional_Person2_prc...
  - duration 48.71
- EXEC dbo.MergeDataDirect_Professional_Person2_prc...


...

 83%|████████▎ | 5/6 [33:33<05:41, 341.35s/it]

  - duration 1.42
flow for target entity Professional_ProToProFunction..
- EXEC dbo.PullData_Professional_ProToProFunction2_prc...
  - duration 47.52
rows 17431418, stage table size: 1109.140625
- EXEC dbo.MergeData_Professional_ProToProFunction2_prc...
  - duration 01:37
- EXEC dbo.MergeDataDirect_Professional_ProToProFunction2_prc...


100%|██████████| 6/6 [38:13<00:00, 382.31s/it]

  - duration 02:15


KeyboardInterrupt: 

In [4]:
columns = ['Entity name', 'Pull SP', 'Merge SP', 'sum', 'Merge Direct',  'rows_count, K', 'table_size, MB', 'velocity, MB/sec']
df = pd.DataFrame(durations, columns = columns)
# df['table_size, MB'] = df ['table_size, MB'].astype('int')
# df['rows_count, K'] = df['rows_count, K']/1000
df

,Entity name,Pull SP,Merge SP,sum,Merge Direct,"rows_count, K","table_size, MB","velocity, MB/sec"
0,Professional_Compensation2,02:44,05:52,08:37,04:15,48222.28,3795.20,7.34
1,Professional_Person2,40.43,48.71,01:29,01:04,5959.98,361.02,4.05
2,Professional_BiographyPerson2,01:23,36.29,01:59,02:15,5367.68,3037.58,25.45
3,Professional_Professional2,03:50,04:45,08:35,05:12,10688.11,1825.59,3.54
4,Professional_ProfessionalCoverage2,0.92,1.13,2.05,1.42,215.35,7.58,3.70
5,Professional_ProToProFunction2,47.52,01:37,02:24,02:15,17431.42,1109.14,7.66
